# Number theory neat examples

<span style="font-size: 16pt; font-style: italic; font-weight: bold">Set 1 : </span>
<span style="font-size: 16pt; font-style: italic;">primes, Klauber triangle, Ulam spiral, sunflower seed spirals</span>

Anton Antonov    
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
January 2025     

In [ ]:
#% js
    (^3).map({
        my $ga = [ϕ, 1.1, 2.8].pick;
        my $color-scheme = <Set1 Set2 Tableau10 Observable10>.pick;

        my @sunflower = (1..1_000).map({
            my $a = $_;
            my $angle = $a * 2 * π / $ga²;
            { x => sqrt($a) * cos($angle), y => sqrt($a) * sin($angle), group => is-prime($a).Int.Str }
        });

        js-d3-list-plot(@sunflower, 
            background => 'none',
            point-size => 4,
            width => 220, height => 212, 
            margins => {:2top, :10bottom, :10left, :10right},
            :!axes, 
            :!legends,
            :$color-scheme
        )    
    }).join("\n")

----

## Introduction

**What is a neat example?** : Concise or straightforward code that produces compelling visual or textual outputs.

**Maybe:** We know *neat* when we see it?

The neat examples:

- Showcase Raku programming.
- Use functionalities of different Raku modules.
- Give interesting perspectives on what is computationally possible.

Showcased:
- All number theory functions are provided by ["Math::NumberTheory"](https://raku.land/zef:antononcube/Math::NumberTheory).   
- Visualization functions are provided by ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3).
- Data manipulation functions are provided by ["Data::Reshapers"](https://raku.land/zef:antononcube/Data::Reshapers).
- Data summarization functions are provided by ["Data::Summarizers"](https://raku.land/zef:antononcube/Data::Summarizers).
- Data translation functions (like `to-html`) are provided by ["Data::Translators"](https://raku.land/zef:antononcube/Data::Translators).

**Remark:** 
- Raku has built-in Number theory functions: `is-prime`, `mod`, `expmod`, `base`, `polymod`. 
- The package "Math::NumberTheory" extends those functions and adds (many) more.

-----

## Setup

In [ ]:
# Part of init.raku
#use Math::NumberTheory;
#use Math::NumberTheory::Utilities;

In [ ]:
%% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [ ]:
#%js
js-d3-list-line-plot(rand xx 40, background => 'none')

-----

## Prime numbers

A table of prime numbers:

In [ ]:
#% html
my $n = 11;

my @highlight = (1..$n²).map({ is-prime($_) ?? $_ !! Empty })».Str;

(1...$n²)
==> to-html(multi-column => $n)
==> html-table-highlight(:@highlight, c => 'Red', w => 'bold')

In [ ]:
# π(x)
prime-pi($n²)

### [Gaussian integers](https://en.wikipedia.org/wiki/Gaussian_integer)

A table of [Gaussian primes](https://en.wikipedia.org/wiki/Gaussian_integer#Gaussian_primes):

In [ ]:
#% html
my $n = 11;

my @highlight = (1..$n²).map({ is-prime($_, :gaussian-integers) ?? $_ !! Empty })».Str;

(1...$n²)
==> to-html(multi-column => $n)
==> html-table-highlight(:@highlight, c => 'Red', w => 'bold')

The table above might look boring -- for any Gaussian prime $p$ holds: $p \mod 4 = 3$.

In [ ]:
(^100).grep(*.&is-prime(:gaussian-integers)).map(* mod 4)

Follows from the [Sum of two squares theorem](https://en.wikipedia.org/wiki/Sum_of_two_squares_theorem).
Relates to [Fermat's theorem on sums of two squares](https://en.wikipedia.org/wiki/Fermat%27s_theorem_on_sums_of_two_squares).

For example, $13 = 2² + 3² = (2 + 3i) (2 - 3i)$, hence $13$ is not a Gaussian prime.

**Remark:** A separate presentation with neat examples over Gaussian integers is planned.

-----

## [Klauber triangle](https://en.wikipedia.org/wiki/Laurence_Monroe_Klauber#Mathematics)

In the early 1930s, herpetologist [Laurence Klauber](https://en.wikipedia.org/wiki/Laurence_Monroe_Klauber) proposed a geometric arrangement of primes. 
(Similar to [Ulam's spiral](https://en.wikipedia.org/wiki/Ulam_spiral), discussed below.)

In [ ]:
#%html
my $n = 11;
my $mat = triangle-matrix-embedding($n, missing-value => ' ', :dataset);

$mat
==> to-html(field-names => (^$mat.head.elems)».Str)
==> html-table-highlight(h => (1..prime-pi($n²))».&prime».Str, c => 'Lime', w => 'bold')

In [ ]:
#% js
my @mat = triangle-matrix-embedding(101, :!dataset).deepmap({ is-prime($_) ?? 2 !! 0 });
js-d3-matrix-plot(@mat, width => 800, height => 400, :!grid-lines, color-palette => 'Magma', :!tooltip)

In [ ]:
#% js
my $k = 13;
my $color-palette = <Magma Cool YlOrRd>.pick;
    my @mat = triangle-matrix-embedding($k, :!dataset).deepmap({ is-prime($_) ?? $_ !! 0 });

    my $smat1 = Math::SparseMatrix.new(dense-matrix => @mat);
    my @data1 = |rename-columns($smat1.tuples(:dataset), (<i j x> Z=> <x y z>).Hash);

    my $smat2 = $smat1[$smat1.row-names.reverse;*];
    my @data2 = |rename-columns($smat2.tuples(:dataset), (<i j x> Z=> <x y z>).Hash);

    js-d3-matrix-plot(@data1, width => 400, height => 300, :!grid-lines, :$color-palette, :!tooltip)
    ~
    js-d3-matrix-plot(@data2, width => 400, height => 300, :!grid-lines, :$color-palette, :!tooltip)

------

## [Ulam spiral](https://en.wikipedia.org/wiki/Ulam_spiral)

The Ulam spiral, also known as the prime spiral, is a visual representation of prime numbers created by mathematician Stanisław Ulam in 1963. It gained popularity through [Martin Gardner](https://en.wikipedia.org/wiki/Martin_Gardner)'s [Mathematical Games column](https://en.wikipedia.org/wiki/Martin_Gardner#Mathematical_Games_column) in Scientific American shortly after. 
The spiral is formed by arranging positive integers in a square spiral and highlighting the prime numbers.

**Remark:** Large fraction of the neat examples I plan to present use Ulam spiral.

Spiral lattice with highlighted primes:

In [ ]:
#% html
my $n = 11;
spiral-lattice($n, end-corner => 'bottom-right', :dataset)
==> to-html(field-names => (^$n)».Str)
==> html-table-highlight(h => (1..prime-pi($n²))».&prime».Str, c => 'Lime', w => 'bold')

Ulam spiral table:

In [ ]:
#% html
spiral-lattice($n, :dataset)
==> { $_.deepmap({ is-prime($_) ?? $_ !! '' }) }()
==> to-html(field-names => (^$n)».Str)

In [ ]:
#% js
my @mat = spiral-lattice(101).deepmap({ is-prime($_) ?? $_ !! 0 });
js-d3-matrix-plot(@mat, width => 400, height => 400, :!grid-lines, color-palette => 'Viridis', :!tooltip)

**Remark:** There are so called ["prime generating polynomials"](https://en.wikipedia.org/wiki/Formula_for_primes#Prime_formulas_and_polynomial_functions).

Diagonals for $4 x^2 - 2 x + 41$:

In [ ]:
#% js
my $n = 201;
my @highlight = (1..$n).map({ 4 * $_ ** 2 - 2 * $_ + 41 })».Int;
my @mat = spiral-lattice($n).deepmap({ is-prime($_) ?? ($_ ∈ @highlight ?? 3 !! 1) !! 0 });
js-d3-matrix-plot(@mat, width => 400, height => 400, :!grid-lines, color-palette => 'Viridis', :!tooltip)

-----

## Next prime

The function `next-prime(x)` gives the smallest prime above $x$.

In [ ]:
next-prime(14, 20)

**Remark:** The following functions are closely related: `is-prime`, `next-prime`, `prime-pi`.

Highlight the sequence of primes using `next-prime`:

In [ ]:
#% html
my $n = 70;

my @highlight = (1..$n).map({ $_ == next-prime($_ - 1) ?? $_ !! Empty })».Str;

(1...$n)
==> to-html(:10multi-column)
==> html-table-highlight(:@highlight, c => 'Red', w => 'bold')

Visualize the [sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes) for the first 5 primes:

In [ ]:
#% js
my @data = 
(0, 0.1 ... 30).map( -> $x { 
                         (1..10).map( -> $n { %( 
                                group => next-prime($n), 
                                :$x, 
                                y => next-prime($n) * sin($x * π / next-prime($n))
                            ) 
                        }) 
                }).flat(1);

js-d3-list-line-plot(@data, background => 'none')

Plot a version of [Ulam spiral](https://en.wikipedia.org/wiki/Ulam_spiral) of the distance to the next prime:

In [ ]:
#% js
my @mat = spiral-lattice(101).deepmap({ next-prime($_) - $_ });
js-d3-matrix-plot(@mat, width => 400, height => 400, :!grid-lines, color-palette => 'Turbo', :!tooltip)

### Sunflower pattern

Sunflower seed representation of `next-prime`:

In [ ]:
my @sunflower = (1..15_000).map({ .&next-prime }).unique.map({
    my $a = $_;
    my $angle = $a * 2 * π / ϕ²;
    { x => sqrt($a) * cos($angle), y => sqrt($a) * sin($angle), group => ($a mod 4).Str }
});

deduce-type(@sunflower)

**Remark:** $360^{\circ} \frac{1}{\phi^2}$ is the [Golden angle](https://en.wikipedia.org/wiki/Golden_angle).
Explanations how it manifests in sunflowers can be found in ["Sunflowers and Fibonacci: Models of Efficiency"](https://thatsmaths.com/2014/06/05/sunflowers-and-fibonacci-models-of-efficiency/).

In [ ]:
360 / ϕ²
==> {.round(.001)}()

So, `next-prime` was used categorize the numbers. (E.g., using `unique` would drastically reduce the number of the plot points.)

In [ ]:
#% js
js-d3-list-plot(@sunflower, 
    background => 'none',
    point-size => 4,
    width => 450, height => 450, 
    :!axes, 
    :!legends,
    color-scheme => 'Set1',
    :5margins
)

----

## Prime Ω

- The function `prime-omega` gives the number of prime factors of an integer with multiplicity.

- For a number $n=u p_1^{k_1}\cdots  p_m^{k_m}$ with $u$ a unit and $p_i$ primes, `prime-omega(n)` returns $k_1 + ... + k_m$.

<table border="1">
  <tbody>
    <tr>
      <th>n</th>
      <td>1</td>
      <td>2</td>
      <td>3</td>
      <td>4</td>
      <td>5</td>
      <td>6</td>
      <td>7</td>
      <td>8</td>
      <td>9</td>
      <td>10</td>
    </tr>
    <tr>
      <th>Ω(n)</th>
      <td>0</td>
      <td>1</td>
      <td>1</td>
      <td>2</td>
      <td>1</td>
      <td>2</td>
      <td>1</td>
      <td>3</td>
      <td>2</td>
      <td>2</td>
    </tr>
  </tbody>
</table>

Make a sunflower based on $\Omega(n)$:

In [ ]:
my @sunflower-omega = (2..15_000).map({
    my $a = $_;
    my $angle = $a * 2 * π / ϕ²;
    my $c = prime-omega($a);
    my $r =  1 / $c; 
    { x => sqrt($a) * cos($angle), y => sqrt($a) * sin($angle), z => $r }
});

deduce-type(@sunflower-omega)

In [ ]:
sink records-summary(@sunflower-omega, field-names => <x y z>)

Plot the sunflower as a bubble chart:

In [ ]:
#% js
my @sunflower-omega2 = @sunflower-omega.grep({ -100 ≤ $_<x> ≤ 100 && -70 ≤ $_<y> ≤ 70 });
js-d3-bubble-chart(@sunflower-omega2,
    z-range-min => 0.8, z-range-max => 3,
    background => 'none',
    width => 600, height => 600 * 0.7, 
    :!axes, 
    :!legends,
    color-scheme => 'Dark2',
    fill-color => 'DarkGray',
    :5margins,
 )

This is a modified version of the [spiral code by Jari Kirma here](https://community.wolfram.com/groups/-/m/t/102049) and [here](https://community.wolfram.com/groups/-/m/t/1543056).